In [1]:
import os
import sys
import argparse

import torch
from torch import nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
from torchvision import datasets
from torchvision import transforms as pth_transforms
from torchvision import models as torchvision_models

import utils
import vision_transformer as vits
from vision_transformer import DINOHead
from my_utils import ResNetMNIST, plot_embed, DINOLoss, train_one_epoch, load_pretrained_backbone_head, CustomWrapper

from torch.utils.data.dataset import Subset

In [65]:
[s[-1] for s in dataset_train.__getitems__(list(range(len(dataset_train))))]

[4,
 1,
 3,
 8,
 8,
 9,
 1,
 0,
 7,
 5,
 5,
 8,
 5,
 1,
 3,
 1,
 1,
 7,
 3,
 5,
 9,
 2,
 5,
 9,
 2,
 2,
 6,
 9,
 3,
 3,
 9,
 6,
 2,
 6,
 0,
 2,
 7,
 5,
 5,
 7,
 7,
 4,
 3,
 6,
 0,
 1,
 8,
 1,
 7,
 1,
 6,
 6,
 0,
 7,
 7,
 5,
 2,
 5,
 5,
 4,
 6,
 4,
 7,
 5,
 9,
 1,
 8,
 2,
 1,
 5,
 7,
 4,
 0,
 5,
 3,
 9,
 3,
 7,
 0,
 8,
 2,
 3,
 0,
 0,
 4,
 3,
 6,
 1,
 3,
 4,
 4,
 1,
 3,
 9,
 7,
 1,
 2,
 9,
 7,
 2,
 5,
 1,
 6,
 3,
 8,
 9,
 0,
 1,
 3,
 6,
 5,
 0,
 4,
 0,
 7,
 0,
 1,
 3,
 6,
 4,
 2,
 2,
 6,
 2,
 1,
 4,
 0,
 2,
 4,
 1,
 0,
 8,
 6,
 9,
 8,
 5,
 2,
 6,
 3,
 9,
 0,
 5,
 2,
 7,
 1,
 8,
 6,
 0,
 3,
 7,
 3,
 6,
 8,
 3,
 9,
 9,
 2,
 5,
 1,
 7,
 7,
 5,
 8,
 1,
 0,
 5,
 3,
 3,
 2,
 5,
 0,
 7,
 5,
 5,
 9,
 4,
 2,
 5,
 1,
 5,
 8,
 6,
 0,
 4,
 4,
 7,
 3,
 1,
 3,
 1,
 0,
 5,
 3,
 9,
 9,
 1,
 4,
 3,
 7,
 7,
 3,
 5,
 4,
 8,
 7,
 4,
 2,
 1,
 5,
 7,
 4,
 2,
 8,
 9,
 6,
 4,
 0,
 8,
 2,
 8,
 4,
 8,
 6,
 9,
 2,
 3,
 2,
 9,
 5,
 7,
 5,
 7,
 6,
 4,
 5,
 2,
 1,
 4,
 3,
 1,
 7,
 5,
 2,
 5,
 9,
 6,
 0,
 3,
 3,
 0,


In [66]:
from torch.utils.data.dataset import Subset
# args.port=str(29500)
# utils.init_distributed_mode(args)
# print("git:\n  {}\n".format(utils.get_sha()))
# print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
# cudnn.benchmark = True

transform = pth_transforms.Compose([
        pth_transforms.ToTensor(),
    ])
class ReturnIndexDataset():
    def __init__(self, dataset):
        self.dataset=dataset
    def __getitem__(self, idx):
        img, lab = self.dataset.__getitem__(idx)
        return img, idx
    def __len__(self):
        return len(self.dataset)
dataset = datasets.MNIST(root="../datasets", download=False, train=False, transform=transform)
dataset_train, dataset_val = torch.utils.data.random_split(dataset, [0.7, 0.3])
index_datset_train = ReturnIndexDataset(dataset_train)
index_datset_val = ReturnIndexDataset(dataset_val)

# dataset_train = ReturnIndexDataset(os.path.join(args.data_path, "train"), transform=transform)
# dataset_val = ReturnIndexDataset(os.path.join(args.data_path, "val"), transform=transform)
sampler = torch.utils.data.DistributedSampler(index_datset_train, shuffle=False)
data_loader_train = torch.utils.data.DataLoader(
    index_datset_train,
    sampler=sampler,
    batch_size=args.batch_size_per_gpu,
    num_workers=args.num_workers,
    pin_memory=True,
    drop_last=False,
)
data_loader_val = torch.utils.data.DataLoader(
    index_datset_val,
    batch_size=args.batch_size_per_gpu,
    num_workers=args.num_workers,
    pin_memory=True,
    drop_last=False,
)
print(f"Data loaded with {len(index_datset_train)} train and {len(index_datset_val)} val imgs.")
args.emb_dim = 10

# ============ building network ... ============
if "resnet" in args.arch.lower():
    embed_dim = args.emb_dim
    model = ResNetMNIST(num_blocks=[2, 2, 2, 2], num_classes=embed_dim)
    print(f"Model ResNet Built")
elif "vit" in args.arch:
    model = vits.__dict__[args.arch](patch_size=args.patch_size, num_classes=0)
    print(f"Model {args.arch} {args.patch_size}x{args.patch_size} built.")
elif "xcit" in args.arch:
    model = torch.hub.load('facebookresearch/xcit:main', args.arch, num_classes=0)
elif args.arch in torchvision_models.__dict__.keys():
    model = torchvision_models.__dict__[args.arch](num_classes=0)
    model.fc = nn.Identity()
else:
    print(f"Architecture {args.arch} non supported")
    sys.exit(1)
model.cuda()
utils.load_pretrained_weights(model, args.pretrained_weights, args.checkpoint_key, args.arch, args.patch_size)
model.eval()

# ============ extract features ... ============
print("Extracting features for train set...")
train_features = extract_features(model, data_loader_train, args.use_cuda)
print("Extracting features for val set...")
test_features = extract_features(model, data_loader_val, args.use_cuda)

if utils.get_rank() == 0:
    train_features = nn.functional.normalize(train_features, dim=1, p=2)
    test_features = nn.functional.normalize(test_features, dim=1, p=2)

# train_labels = torch.tensor([s[-1] for s in dataset_train.samples]).long()
# test_labels = torch.tensor([s[-1] for s in dataset_val.samples]).long()
train_labels = torch.tensor([s[-1] for s in dataset_train.__getitems__(list(range(len(dataset_train))))]).long()
test_labels = torch.tensor([s[-1] for s in dataset_val.__getitems__(list(range(len(dataset_val))))]).long()

# save features and labels
if args.dump_features and dist.get_rank() == 0:
    torch.save(train_features.cpu(), os.path.join(args.dump_features, "trainfeat.pth"))
    torch.save(test_features.cpu(), os.path.join(args.dump_features, "testfeat.pth"))
    torch.save(train_labels.cpu(), os.path.join(args.dump_features, "trainlabels.pth"))
    torch.save(test_labels.cpu(), os.path.join(args.dump_features, "testlabels.pth"))

Data loaded with 7000 train and 3000 val imgs.
Model ResNet Built
Take key teacher in provided checkpoint dict
Pretrained weights found at /workspace/puae/dino/outputs/emb10_arcface_m0.1_warmup0/checkpoints/checkpoint.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])
Extracting features for train set...
Storing features into tensor of shape torch.Size([7000, 10])
  [ 0/70]  eta: 0:00:25    time: 0.362045  data: 0.341849  max mem: 269
  [10/70]  eta: 0:00:02    time: 0.036043  data: 0.031195  max mem: 269
  [20/70]  eta: 0:00:01    time: 0.003599  data: 0.000117  max mem: 269
  [30/70]  eta: 0:00:00    time: 0.004344  data: 0.000160  max mem: 269
  [40/70]  eta: 0:00:00    time: 0.004087  data: 0.000167  max mem: 269
  [50/70]  eta: 0:00:00    time: 0.004062  data: 0.000118  max mem: 26

In [2]:
def extract_feature_pipeline(args):
    # ============ preparing data ... ============
    transform = pth_transforms.Compose([
        # pth_transforms.Resize(256, interpolation=3),
        # pth_transforms.CenterCrop(224),
        pth_transforms.ToTensor(),
        pth_transforms.Normalize((0.1307,), (0.3081,)),
    ])

    dataset = datasets.MNIST(root="../datasets", download=False, train=False, transform=transform)
    dataset_train, dataset_val = torch.utils.data.random_split(dataset, [0.7, 0.3])
    index_datset_train = ReturnIndexDataset(dataset_train)
    index_datset_val = ReturnIndexDataset(dataset_val)

    # dataset_train = ReturnIndexDataset(os.path.join(args.data_path, "train"), transform=transform)
    # dataset_val = ReturnIndexDataset(os.path.join(args.data_path, "val"), transform=transform)
    sampler = torch.utils.data.DistributedSampler(index_datset_train, shuffle=False)
    data_loader_train = torch.utils.data.DataLoader(
        index_datset_train,
        sampler=sampler,
        batch_size=args.batch_size_per_gpu,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
    data_loader_val = torch.utils.data.DataLoader(
        index_datset_val,
        batch_size=args.batch_size_per_gpu,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=False,
    )
    print(f"Data loaded with {len(index_datset_train)} train and {len(index_datset_val)} val imgs.")

    # ============ building network ... ============
    if "resnet" in args.arch.lower():
        embed_dim = args.emb_dim
        if "head" in args.arch.lower():
            teacher = ResNetMNIST(num_blocks=[2, 2, 2, 2], num_classes=embed_dim)
            model = CustomWrapper(
            teacher,
            DINOHead(embed_dim, 2000, False, arcface_family_conf={"name":False}),
        )
        else:
            model = ResNetMNIST(num_blocks=[2, 2, 2, 2], num_classes=embed_dim)
        print(f"Model ResNet emb_dim:{args.emb_dim} Built")
    elif "vit" in args.arch:
        model = vits.__dict__[args.arch](patch_size=args.patch_size, num_classes=0)
        print(f"Model {args.arch} {args.patch_size}x{args.patch_size} built.")
    elif "xcit" in args.arch:
        model = torch.hub.load('facebookresearch/xcit:main', args.arch, num_classes=0)
    elif args.arch in torchvision_models.__dict__.keys():
        model = torchvision_models.__dict__[args.arch](num_classes=0)
        model.fc = nn.Identity()
    else:
        print(f"Architecture {args.arch} non supported")
        sys.exit(1)
    model.cuda()
    if "resnet" in args.arch.lower() and "head" in args.arch.lower():
        load_pretrained_backbone_head(model, args.pretrained_weights, "teacher")
        print(f"load resnet backbone and head. Then, delete last layer")
    else:
        utils.load_pretrained_weights(model, args.pretrained_weights, args.checkpoint_key, args.arch, args.patch_size)
    model.eval()

    # ============ extract features ... ============
    print("Extracting features for train set...")
    train_features = extract_features(model, data_loader_train, args.use_cuda)
    print("Extracting features for val set...")
    test_features = extract_features(model, data_loader_val, args.use_cuda)

    if utils.get_rank() == 0:
        train_features = nn.functional.normalize(train_features, dim=1, p=2)
        test_features = nn.functional.normalize(test_features, dim=1, p=2)

    # train_labels = torch.tensor([s[-1] for s in dataset_train.samples]).long()
    # test_labels = torch.tensor([s[-1] for s in dataset_val.samples]).long()
    train_labels = torch.tensor([s[-1] for s in dataset_train.__getitems__(list(range(len(dataset_train))))]).long()
    test_labels = torch.tensor([s[-1] for s in dataset_val.__getitems__(list(range(len(dataset_val))))]).long()

    # save features and labels
    if args.dump_features and dist.get_rank() == 0:
        torch.save(train_features.cpu(), os.path.join(args.dump_features, "trainfeat.pth"))
        torch.save(test_features.cpu(), os.path.join(args.dump_features, "testfeat.pth"))
        torch.save(train_labels.cpu(), os.path.join(args.dump_features, "trainlabels.pth"))
        torch.save(test_labels.cpu(), os.path.join(args.dump_features, "testlabels.pth"))
    return train_features, test_features, train_labels, test_labels


@torch.no_grad()
def extract_features(model, data_loader, use_cuda=True, multiscale=False):
    metric_logger = utils.MetricLogger(delimiter="  ")
    features = None
    for samples, index in metric_logger.log_every(data_loader, 10):
        samples = samples.cuda(non_blocking=True)
        index = index.cuda(non_blocking=True)
        if multiscale:
            feats = utils.multi_scale(samples, model)
        else:
            feats = model(samples).clone()

        # init storage feature matrix
        if dist.get_rank() == 0 and features is None:
            features = torch.zeros(len(data_loader.dataset), feats.shape[-1])
            if use_cuda:
                features = features.cuda(non_blocking=True)
            print(f"Storing features into tensor of shape {features.shape}")

        # get indexes from all processes
        y_all = torch.empty(dist.get_world_size(), index.size(0), dtype=index.dtype, device=index.device)
        y_l = list(y_all.unbind(0))
        y_all_reduce = torch.distributed.all_gather(y_l, index, async_op=True)
        y_all_reduce.wait()
        index_all = torch.cat(y_l)

        # share features between processes
        feats_all = torch.empty(
            dist.get_world_size(),
            feats.size(0),
            feats.size(1),
            dtype=feats.dtype,
            device=feats.device,
        )
        output_l = list(feats_all.unbind(0))
        output_all_reduce = torch.distributed.all_gather(output_l, feats, async_op=True)
        output_all_reduce.wait()

        # update storage feature matrix
        if dist.get_rank() == 0:
            if use_cuda:
                features.index_copy_(0, index_all, torch.cat(output_l))
            else:
                features.index_copy_(0, index_all.cpu(), torch.cat(output_l).cpu())
    return features


@torch.no_grad()
def knn_classifier(train_features, train_labels, test_features, test_labels, k, T, num_classes=1000):
    top1, top5, total = 0.0, 0.0, 0
    train_features = train_features.t()
    num_test_images, num_chunks = test_labels.shape[0], 100
    imgs_per_chunk = num_test_images // num_chunks
    retrieval_one_hot = torch.zeros(k, num_classes).to(train_features.device)
    for idx in range(0, num_test_images, imgs_per_chunk):
        # get the features for test images
        features = test_features[
            idx : min((idx + imgs_per_chunk), num_test_images), :
        ]
        targets = test_labels[idx : min((idx + imgs_per_chunk), num_test_images)]
        batch_size = targets.shape[0]

        # calculate the dot product and compute top-k neighbors
        similarity = torch.mm(features, train_features)
        distances, indices = similarity.topk(k, largest=True, sorted=True)
        candidates = train_labels.view(1, -1).expand(batch_size, -1)
        retrieved_neighbors = torch.gather(candidates, 1, indices)

        retrieval_one_hot.resize_(batch_size * k, num_classes).zero_()
        retrieval_one_hot.scatter_(1, retrieved_neighbors.view(-1, 1), 1)
        distances_transform = distances.clone().div_(T).exp_()
        probs = torch.sum(
            torch.mul(
                retrieval_one_hot.view(batch_size, -1, num_classes),
                distances_transform.view(batch_size, -1, 1),
            ),
            1,
        )
        _, predictions = probs.sort(1, True)

        # find the predictions that match the target
        correct = predictions.eq(targets.data.view(-1, 1))
        top1 = top1 + correct.narrow(1, 0, 1).sum().item()
        top5 = top5 + correct.narrow(1, 0, min(5, k)).sum().item()  # top5 does not make sense if k < 5
        total += targets.size(0)
    top1 = top1 * 100.0 / total
    top5 = top5 * 100.0 / total
    return top1, top5


# class ReturnIndexDataset(datasets.ImageFolder):
#     def __getitem__(self, idx):
#         img, lab = super(ReturnIndexDataset, self).__getitem__(idx)
#         return img, idx

class ReturnIndexDataset():
    def __init__(self, dataset):
        self.dataset=dataset
    def __getitem__(self, idx):
        img, lab = self.dataset.__getitem__(idx)
        return img, idx
    def __len__(self):
        return len(self.dataset)

In [10]:
# parser = argparse.ArgumentParser('Evaluation with weighted k-NN on ImageNet')
# parser.add_argument('--batch_size_per_gpu', default=128, type=int, help='Per-GPU batch-size')
# parser.add_argument('--nb_knn', default=[10, 20, 100, 200], nargs='+', type=int,
#     help='Number of NN to use. 20 is usually working the best.')
# parser.add_argument('--temperature', default=0.07, type=float,
#     help='Temperature used in the voting coefficient')
# parser.add_argument('--pretrained_weights', default='', type=str, help="Path to pretrained weights to evaluate.")
# parser.add_argument('--use_cuda', default=True, type=utils.bool_flag,
#     help="Should we store the features on GPU? We recommend setting this to False if you encounter OOM")
# parser.add_argument('--arch', default='vit_small', type=str, help='Architecture')
# parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
# parser.add_argument("--checkpoint_key", default="teacher", type=str,
#     help='Key to use in the checkpoint (example: "teacher")')
# parser.add_argument('--dump_features', default=None,
#     help='Path where to save computed features, empty for no saving')
# parser.add_argument('--load_features', default=None, help="""If the features have
#     already been computed, where to find them.""")
# parser.add_argument('--num_workers', default=10, type=int, help='Number of data loading workers per GPU.')
# parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set up
#     distributed training; see https://pytorch.org/docs/stable/distributed.html""")
# parser.add_argument("--local_rank", default=0, type=int, help="Please ignore and do not set this argument.")
# parser.add_argument('--data_path', default='/path/to/imagenet/', type=str)
# args = parser.parse_args()

utils.init_distributed_mode(args)
print("git:\n  {}\n".format(utils.get_sha()))
print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
cudnn.benchmark = True

if args.load_features:
    train_features = torch.load(os.path.join(args.load_features, "trainfeat.pth"))
    test_features = torch.load(os.path.join(args.load_features, "testfeat.pth"))
    train_labels = torch.load(os.path.join(args.load_features, "trainlabels.pth"))
    test_labels = torch.load(os.path.join(args.load_features, "testlabels.pth"))
else:
    # need to extract features !
    train_features, test_features, train_labels, test_labels = extract_feature_pipeline(args)

if utils.get_rank() == 0:
    if args.use_cuda:
        train_features = train_features.cuda()
        test_features = test_features.cuda()
        train_labels = train_labels.cuda()
        test_labels = test_labels.cuda()

    print("Features are ready!\nStart the k-NN classification.")
    for k in args.nb_knn:
        top1, top5 = knn_classifier(train_features, train_labels,
            test_features, test_labels, k, args.temperature)
        print(f"{k}-NN classifier result: Top1: {top1}, Top5: {top5}")
dist.barrier()

usage: Evaluation with weighted k-NN on ImageNet [-h]
                                                 [--batch_size_per_gpu BATCH_SIZE_PER_GPU]
                                                 [--nb_knn NB_KNN [NB_KNN ...]]
                                                 [--temperature TEMPERATURE]
                                                 [--pretrained_weights PRETRAINED_WEIGHTS]
                                                 [--use_cuda USE_CUDA]
                                                 [--arch ARCH]
                                                 [--patch_size PATCH_SIZE]
                                                 [--checkpoint_key CHECKPOINT_KEY]
                                                 [--dump_features DUMP_FEATURES]
                                                 [--load_features LOAD_FEATURES]
                                                 [--num_workers NUM_WORKERS]
                                                 [--dist_url DIST_URL]
     

SystemExit: 2

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
class my_args:
        pass

In [11]:
args=my_args
args.batch_size_per_gpu = 100
args.nb_knn = [10, 20, 100, 200]
args.temperature = 0.07
args.pretrained_weights = '/workspace/puae/dino/outputs/emb10_arcface_m0.3_warmup10/checkpoints/checkpoint.pth'
args.use_cuda = True
args.arch = "ResNet head"
args.patch_size = 16
args.checkpoint_key = "teacher"
args.dump_features = None
args.load_features = None
args.num_workers = 10
args.dist_url = "env://"
args.local_rank = 0
args.data_path = ""
args.emb_dim = 10
args.out_filename = "head_knn_result.txt"
args.port = "29500"

In [12]:
# utils.init_distributed_mode(args)
# print("git:\n  {}\n".format(utils.get_sha()))
# print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
# cudnn.benchmark = True

if args.load_features:
    train_features = torch.load(os.path.join(args.load_features, "trainfeat.pth"))
    test_features = torch.load(os.path.join(args.load_features, "testfeat.pth"))
    train_labels = torch.load(os.path.join(args.load_features, "trainlabels.pth"))
    test_labels = torch.load(os.path.join(args.load_features, "testlabels.pth"))
else:
    # need to extract features !
    train_features, test_features, train_labels, test_labels = extract_feature_pipeline(args)

if utils.get_rank() == 0:
    if args.use_cuda:
        train_features = train_features.cuda()
        test_features = test_features.cuda()
        train_labels = train_labels.cuda()
        test_labels = test_labels.cuda()

    results={}
    print("Features are ready!\nStart the k-NN classification.")
    for k in args.nb_knn:
        top1, top5 = knn_classifier(train_features, train_labels,
            test_features, test_labels, k, args.temperature)
        print(f"{k}-NN classifier result: Top1: {top1}, Top5: {top5}")
        results[f"{k}-NN"] = {"Top1":top1, "Top5":top5}
dist.barrier()

log_file_pth = os.path.dirname(os.path.dirname(args.pretrained_weights))
if not os.path.exists(os.path.join(log_file_pth, os.path.join(log_file_pth, args.out_filename))):
    with open(os.path.join(log_file_pth, args.out_filename), mode="x") as f:
        f.write(f"{args.pretrained_weights}\n")
        f.write(f"{args.arch}\n")
        for i in results.items():
            f.write(str(i[0])+" "+str(i[1])+"\n")

Data loaded with 7000 train and 3000 val imgs.
Model ResNet emb_dim:10 Built
load resnet backbone and head. Then, delete last layer
Extracting features for train set...
Storing features into tensor of shape torch.Size([7000, 256])
  [ 0/70]  eta: 0:00:27    time: 0.395379  data: 0.374108  max mem: 199
  [10/70]  eta: 0:00:02    time: 0.040870  data: 0.034190  max mem: 199
  [20/70]  eta: 0:00:01    time: 0.004589  data: 0.000156  max mem: 199
  [30/70]  eta: 0:00:00    time: 0.003841  data: 0.000133  max mem: 199
  [40/70]  eta: 0:00:00    time: 0.003635  data: 0.000143  max mem: 199
  [50/70]  eta: 0:00:00    time: 0.004185  data: 0.000141  max mem: 199
  [60/70]  eta: 0:00:00    time: 0.004958  data: 0.000131  max mem: 199
  [69/70]  eta: 0:00:00    time: 0.004937  data: 0.000086  max mem: 199
 Total time: 0:00:00 (0.010780 s / it)
Extracting features for val set...
Storing features into tensor of shape torch.Size([3000, 256])
  [ 0/30]  eta: 0:00:10    time: 0.343802  data: 0.322893

In [84]:
for i in results.items():
    print(str(i[0]), str(i[1]))

10-NN {'Top1': 93.1, 'Top5': 99.6}
20-NN {'Top1': 92.06666666666666, 'Top5': 99.73333333333333}
100-NN {'Top1': 90.26666666666667, 'Top5': 99.83333333333333}
200-NN {'Top1': 89.76666666666667, 'Top5': 99.8}


In [89]:
log_file_pth = os.path.dirname(os.path.dirname(args.pretrained_weights))
with open(os.path.join(log_file_pth, "knn_result.txt"), mode="x") as f:
    f.write(f"{args.pretrained_weights}\n")
    for i in results.items():
        f.write(str(i[0])+" "+str(i[1])+"\n")

In [95]:
os.path.exists(os.path.join(log_file_pth, os.path.join(log_file_pth, "knn_result.txt")))

False